In [6]:
#Importo las librerías que voy a usar a lo largo del ejercicio:
import pandas as pd
# -----------------------------------------------------------------------
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
#import matplotlib.pyplot as plt
#import seaborn as sns

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats

from scipy.stats import shapiro, levene, mannwhitneyu
from scipy.stats import ttest_ind, norm, chi2_contingency

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [7]:
df= pd.read_csv("finanzas-hotel-bookings.csv")


In [8]:
df.head(7)

,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,0,Resort Hotel,False,342.0,2015.0,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,1,Resort Hotel,False,737.0,NaN,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,2,Resort Hotel,False,7.0,2015.0,July,27.0,1.0,0.0,1.0,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,3,Resort Hotel,False,13.0,NaN,July,27.0,1.0,0.0,1.0,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,4,Resort Hotel,False,14.0,NaN,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
5,5,Resort Hotel,False,14.0,NaN,July,27.0,1.0,0.0,2.0,2.0,NaN,0.0,BB,GBR,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,NaN,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
6,6,Resort Hotel,False,0.0,2015.0,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,NaN,NaN,0.0,NaN,107.0,0.0,0.0,Check-Out,2015-07-03 00:00:00,NaN


In [9]:
df["Unnamed: 0"].value_counts()

Unnamed: 0
0         1
121933    1
121913    1
121914    1
121915    1
         ..
60961     1
60962     1
60963     1
60964     1
182876    1
Name: count, Length: 182877, dtype: int64

In [10]:
#Con info me hace un pequeño resumen del df: columnas, tipos de datos, no nulos,...
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182877 entries, 0 to 182876
Data columns (total 33 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      182877 non-null  int64  
 1   hotel                           119390 non-null  object 
 2   is_canceled                     119390 non-null  object 
 3   lead_time                       119490 non-null  float64
 4   arrival_date_year               64829 non-null   float64
 5   arrival_date_month              119390 non-null  object 
 6   arrival_date_week_number        101004 non-null  float64
 7   arrival_date_day_of_month       119271 non-null  float64
 8   stays_in_weekend_nights         119490 non-null  float64
 9   stays_in_week_nights            119490 non-null  float64
 10  adults                          119428 non-null  float64
 11  children                        69302 non-null   float64
 12  babies          

In [11]:
def revision_inicial(dataframe):
    """" ""
    Esta función realiza un análisis exploratorio básico de un DataFrame, mostrando información sobre sus nombres de columnas,
    tamaño general, duplicados, valores nulos, tipos de datos, y estadísticas descriptivas para columnas categóricas y numéricas. 
    """
    
    print(f"Este dataframe cuenta con los siguientes nombres de columnas: \n{dataframe.columns}") 
    """Miro el nombre de las columnas"""

    print("-----\n")
    print(f"Los tipos de datos de sus columnas son:")
    display(pd.DataFrame(dataframe.dtypes, columns = ["Tipo_de_dato"]))

    """Con dtypes miro qué tipo de dato tiene cada columna"""

    print(f"Cuenta con {dataframe.shape[0]} FILAS, y {dataframe.shape[1]} COLUMNAS.")
    """Shape me devuelve una tupla con el numero de filas [0] y columnas [1]"""
    print("-----\n")

    print(f"Estos son los estadísticos de sus columnas numéricas:")
    display(dataframe.describe().T )
    """Con describe veo valores estadísticos de las columnas numéricas como la media, la mediana, la desv. estándar, o el mínimo entre otros."""
    try:
        print("Estos son los estadísticos de sus columnas categóricas:")
        display(dataframe.describe(include="O").T )
        for col in dataframe:
            print(f"La columa {col.upper()} tiene los siguientes valores únicos:\n {dataframe[col].unique()}")

        """Con describe include o de object, busco solo los estadísticos de las columnas categoricas, como sus valores únicos, las frecuencias de estos o cuál es el que más se repite por columna."""
    
    except:
        """Manejo esta excepción porque uno de los dataframes no tiene columnas categóricas."""
        print("NO tiene columnas categóricas.")
    
    print("-----\n")
    print(f"Cuenta con este total de nulos en cada columna:")
    display(pd.DataFrame(dataframe.isnull().sum(), columns= ["Nulos"]))
    
    porcentaje_null= (dataframe.isnull().sum()/df.shape[0]) *100 #hago el cálculo
    nulos_df= pd.DataFrame(porcentaje_null.round(2), columns = ["Nulos_Porcentaje"]) #convierto a dataframe y redondeo las cifras
    display(nulos_df[nulos_df["Nulos_Porcentaje"]>0]) #filtro para que solo me muestre las columnas con más de cero nulos.
    
    print("-----\n")
    print(f"El numero total de DUPLICADOS es: {dataframe.duplicated().sum()}.")
        

In [12]:
revision_inicial(df)

Este dataframe cuenta con los siguientes nombres de columnas: 
Index(['Unnamed: 0', 'hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', '0'],
      dtype='object')
-----

Los tipos de datos de sus columnas son:


,Tipo_de_dato
Unnamed: 0,int64
hotel,object
is_canceled,object
lead_time,float64
arrival_date_year,float64
arrival_date_month,object
arrival_date_week_number,float64
arrival_date_day_of_month,float64
stays_in_weekend_nights,float64
stays_in_week_nights,float64


Cuenta con 182877 FILAS, y 33 COLUMNAS.
-----

Estos son los estadísticos de sus columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,182877.0,91438.000000,52792.186927,0.00,45719.00,91438.000,137157.0,182876.000000
lead_time,119490.0,104.172628,106.975949,0.00,18.00,69.000,161.0,737.000000
arrival_date_year,64829.0,2016.156196,0.706674,2015.00,2016.00,2016.000,2017.0,2017.000000
arrival_date_week_number,101004.0,27.175785,13.613871,1.00,16.00,28.000,38.0,53.000000
arrival_date_day_of_month,119271.0,15.795977,8.780503,1.00,8.00,16.000,23.0,31.000000
stays_in_weekend_nights,119490.0,0.939461,1.082472,0.00,0.00,1.000,2.0,19.914715
stays_in_week_nights,119490.0,2.515068,1.976511,0.00,1.00,2.000,3.0,50.000000
adults,119428.0,6.244423,14.574814,0.00,2.00,2.000,2.0,59.000000
children,69302.0,0.125162,0.688305,0.00,0.00,0.000,0.0,19.000000
babies,119431.0,0.019903,0.433366,0.00,0.00,0.000,0.0,19.000000


Estos son los estadísticos de sus columnas categóricas:


,count,unique,top,freq
hotel,119390,2,City Hotel,79330
is_canceled,119390,2,False,75166
arrival_date_month,119390,15,August,13877
meal,119390,5,BB,92310
country,65054,163,PRT,26577
market_segment,59934,8,Online TA,28331
distribution_channel,105899,5,TA/TO,86822
reserved_room_type,79275,10,A,57202
assigned_room_type,119390,12,A,74053
customer_type,94199,4,Transient,70610


La columa UNNAMED: 0 tiene los siguientes valores únicos:
 [     0      1      2 ... 182874 182875 182876]
La columa HOTEL tiene los siguientes valores únicos:
 ['Resort Hotel' 'City Hotel' nan]
La columa IS_CANCELED tiene los siguientes valores únicos:
 [False True nan]
La columa LEAD_TIME tiene los siguientes valores únicos:
 [342.         737.           7.          13.          14.
   0.           9.          85.          75.          23.
  35.          68.          18.          37.          12.
  72.         127.          78.          48.          60.
  77.          99.         118.          95.          96.
  69.          45.          40.          15.          36.
  43.          70.          16.         107.          47.
 113.          90.          50.          93.          76.
   3.           1.          10.           5.          17.
  51.          71.          63.          62.         101.
   2.          81.         368.         364.         324.
  79.          21.         109. 

,Nulos
Unnamed: 0,0
hotel,63487
is_canceled,63487
lead_time,63387
arrival_date_year,118048
arrival_date_month,63487
arrival_date_week_number,81873
arrival_date_day_of_month,63606
stays_in_weekend_nights,63387
stays_in_week_nights,63387


,Nulos_Porcentaje
hotel,34.72
is_canceled,34.72
lead_time,34.66
arrival_date_year,64.55
arrival_date_month,34.72
arrival_date_week_number,44.77
arrival_date_day_of_month,34.78
stays_in_weekend_nights,34.66
stays_in_week_nights,34.66
adults,34.69


-----

El numero total de DUPLICADOS es: 0.


In [13]:
def tipo_columns(dataframe):
    categoricas= dataframe.select_dtypes(include="O")
    numericas= dataframe.select_dtypes(include= np.number)
    col_numericas= numericas.columns
    col_categoricas= categoricas.columns
    
    for col in col_categoricas: 
        print("Columna categórica:")
        print(col.upper()) #pongo nombre de cada columna en mayus para que se vea más claro
        print(categoricas[col].unique())
        print(categoricas[col].value_counts())
        print(f"Total de duplicados: {categoricas[col].duplicated().sum()}")
        print(f"Total de nulos: {categoricas[col].isnull().sum()}")
        print("------\nSiguiente columna:\n")

    print("Estas son todas las columnas categóricas.") 

    for colu in col_numericas:
        print("Columna numérica:")
        print(colu.upper()) #pongo nomnbre de cada columna en mayus para que se vea más claro
        print(numericas[colu].unique()) 
        print(numericas[colu].value_counts())
        print(f"Total de nulos: {numericas[colu].isnull().sum()}")
        print(f"Total de duplicados: {numericas[colu].duplicated().sum()}")

        print("------\nSiguiente columna:\n")

    print("Estas son todas las columnas numéricas.")


In [14]:
tipo_columns(df)

Columna categórica:
HOTEL
['Resort Hotel' 'City Hotel' nan]
hotel
City Hotel      79330
Resort Hotel    40060
Name: count, dtype: int64
Total de duplicados: 182874
Total de nulos: 63487
------
Siguiente columna:

Columna categórica:
IS_CANCELED
[False True nan]
is_canceled
False    75166
True     44224
Name: count, dtype: int64
Total de duplicados: 182874
Total de nulos: 63487
------
Siguiente columna:

Columna categórica:
ARRIVAL_DATE_MONTH
['July' 'August' 'September' 'October' 'November' 'December' '1' 'January'
 'February' '2' '3' 'March' 'April' 'May' 'June' nan]
arrival_date_month
August       13877
July         12661
May          11791
October      11160
April        11089
June         10939
September    10508
November      6794
December      6780
March         5922
February      4898
3             3872
January       3581
2             3170
1             2348
Name: count, dtype: int64
Total de duplicados: 182861
Total de nulos: 63487
------
Siguiente columna:

Columna categórica

In [15]:
df.columns

Index(['Unnamed: 0', 'hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', '0'],
      dtype='object')

In [16]:
#HACER FILTRO PARA VER LAS RESERVADAS NULAS Y VALORES UNICOS DE LAS ASIGNADAS

In [17]:
#Miro los nulos de lead time, que es una de las columnas con menos porcentaje de nulos
nulos= df[df["lead_time"].isnull()]
nulos.sample(2)

,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
125098,125098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRT
179352,179352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [18]:
#hago copy por siaca
df1 = df.copy()

In [19]:
df.columns

Index(['Unnamed: 0', 'hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', '0'],
      dtype='object')

In [20]:
#borro nulos con la referencia de lead time
df1 = df1.dropna(subset=['lead_time'])

In [21]:
#miro de nuevo porcentajes de nulos
porcentaje_null= (df1.isnull().sum()/df.shape[0]) *100 #hago el cálculo
nulos_dataf= pd.DataFrame(porcentaje_null.round(2), columns = ["Nulos_Porcentaje"]) #convierto a dataframe y redondeo las cifras
display(nulos_dataf[nulos_dataf["Nulos_Porcentaje"]>0]) #filtro para que solo me muestre las columnas con más de cero nulos.

,Nulos_Porcentaje
hotel,0.05
is_canceled,0.05
arrival_date_year,29.89
arrival_date_month,0.05
arrival_date_week_number,10.11
arrival_date_day_of_month,0.12
adults,0.05
children,27.46
babies,0.05
meal,0.05


In [22]:
df1.drop("0", axis=1, inplace=True)

In [18]:
df1

,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,Resort Hotel,False,342.000000,2015.0,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00
1,1,Resort Hotel,False,737.000000,NaN,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00
2,2,Resort Hotel,False,7.000000,2015.0,July,27.0,1.0,0.000000,1.000000,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00
3,3,Resort Hotel,False,13.000000,NaN,July,27.0,1.0,0.000000,1.000000,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00
4,4,Resort Hotel,False,14.000000,NaN,July,NaN,1.0,0.000000,2.000000,2.0,NaN,0.0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182872,182872,NaN,NaN,224.337762,NaN,NaN,NaN,NaN,19.005545,19.819823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182873,182873,NaN,NaN,390.141963,NaN,NaN,NaN,NaN,14.751794,19.989726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182874,182874,NaN,NaN,230.689826,NaN,NaN,NaN,NaN,11.409496,20.461372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182875,182875,NaN,NaN,304.888534,NaN,NaN,NaN,NaN,16.744472,15.400773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#df1.to_csv("finanzas-hotel-bookings-clean.csv")


In [23]:
df["total_of_special_requests"] = df["total_of_special_requests"].astype(str).str.replace(".", "")

In [24]:
df2= df1.copy()

In [25]:
df2["babies"]= df2["babies"].astype(str).str.split(".").str[0]

In [26]:
df2["babies"]= df2["babies"].replace("nan", "")

In [27]:
# df2["babies"] = df2["babies"].replace("", np.number) ··· no me funciona lo cambio
df2["babies"].replace("", np.nan, inplace=True) 

In [28]:
df2["babies"].unique()

array(['0', '1', '18', '2', '17', '13', '15', '16', '12', '10', '11',
       '19', '9', '14', nan], dtype=object)

In [29]:
df2['babies'] = pd.to_numeric(df2['babies'], errors='coerce').astype('Int64')


In [30]:
listado = ['lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company', 'days_in_waiting_list', 'required_car_parking_spaces', 'total_of_special_requests']

In [31]:
def quitar_puntos(df, lista):
   for col in lista:
    # Convierto a cadena, spliteo por el punto y reasigno a la columna del DataFrame
    df[col] = df[col].astype(str).str.split(".").str[0].replace("nan", "").replace("", np.nan)
    # Convierte la columna numérica forzando a NaN en caso de error ("coerce")
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')


In [29]:
# def quitar_puntos(df, lista):
#     for col in lista:
#        # Convierto a cadena, spliteo por el punto y reasigno a la columna del DataFrame
#         df[col] = df[col].astype(str).str.split(".").str[0].replace("nan", "").replace("", np.number)
#         #convierte la columna numérica forzando a NaN en el caso de error ("coerce")
#         df[col]= pd.to_numeric(df[col], errors='coerce').astype('Int64')
         
#     return df

In [32]:
quitar_puntos(df1, listado)

In [33]:
df1[df1["hotel"].isnull()]


,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
182777,182777,NaN,NaN,285,<NA>,NaN,<NA>,<NA>,11,19,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN
182778,182778,NaN,NaN,259,<NA>,NaN,<NA>,<NA>,17,24,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN
182779,182779,NaN,NaN,350,<NA>,NaN,<NA>,<NA>,18,21,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN
182780,182780,NaN,NaN,249,<NA>,NaN,<NA>,<NA>,10,22,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN
182781,182781,NaN,NaN,299,<NA>,NaN,<NA>,<NA>,16,20,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182872,182872,NaN,NaN,224,<NA>,NaN,<NA>,<NA>,19,19,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN
182873,182873,NaN,NaN,390,<NA>,NaN,<NA>,<NA>,14,19,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN
182874,182874,NaN,NaN,230,<NA>,NaN,<NA>,<NA>,11,20,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN
182875,182875,NaN,NaN,304,<NA>,NaN,<NA>,<NA>,16,15,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,NaN


In [34]:
df1 = df1.dropna(subset=['hotel'])

In [35]:
df1["hotel"].isnull().sum()

0

In [36]:
#miro de nuevo porcentajes de nulos
porcentaje_null= (df1.isnull().sum()/df1.shape[0]) *100 #hago el cálculo
nulos_dataf= pd.DataFrame(porcentaje_null.round(2), columns = ["Nulos_Porcentaje"]) #convierto a dataframe y redondeo las cifras
display(nulos_dataf[nulos_dataf["Nulos_Porcentaje"]>0]) #filtro para que solo me muestre las columnas con más de cero nulos.

,Nulos_Porcentaje
arrival_date_year,45.70
arrival_date_week_number,15.40
arrival_date_day_of_month,0.10
children,41.98
country,45.51
market_segment,49.80
distribution_channel,11.30
is_repeated_guest,4.20
previous_cancellations,36.18
reserved_room_type,33.60


In [37]:
#vemos que es booleano
df1["is_canceled"].unique()

array([False, True], dtype=object)

In [38]:
#Cambiamos valores 1, 2 y 3 por el nombre del mes en columna arrivaldatemonth
df1["arrival_date_month"]= df1["arrival_date_month"].str.replace("1", "January").str.replace("2", "February").str.replace("3", "March")

In [39]:
df1["arrival_date_month"].unique()

array(['July', 'August', 'September', 'October', 'November', 'December',
       'January', 'February', 'March', 'April', 'May', 'June'],
      dtype=object)

In [40]:
#hacerle más adelante undefined a NAN
df1["customer_type"].unique()

array(['Transient', nan, 'Contract', 'Transient-Party', 'Group'],
      dtype=object)

In [41]:
df1["is_repeated_guest"] 

0         0
1         0
2         0
3         0
4         0
         ..
119385    0
119386    0
119387    0
119388    0
119389    0
Name: is_repeated_guest, Length: 119390, dtype: Int64

In [42]:
df1["is_repeated_guest"] = df1["is_repeated_guest"].astype(str).str.replace("1", "True").str.replace("0", "False") # hay que poner filtro sale don false

In [41]:
#Pasamos esta columna a tipo fecha
#df1["reservation_status_date"]= pd.to_datetime(df1["reservation_status_date"], format='%Y-%m-%d', errors='coerce')

In [42]:
#df1.to_csv("finanzas-hotel-bookings-clean.csv")

In [43]:
df1["country"].isnull().sum()

54336

In [44]:
#A comprobar categoricas:
#['country', 'market_segment', 'customer_type', 'reservation_status',  'distribution_channel']

#Borrar company


arrival date month cambiar nums por nombres y cambiar a objeto

market segment debería ser object, es int, cambiar undefined por la mediana? no, muchos nulos 122943
distrib channel es int tiene que ser object cambiar nan por undefined?
country es int tiene que ser object
cubrir assigned room type y reserved room type por faltantes, pasar int a object en ambas

reservation status date pasar a date, algunas no tienen hora

arrival date year quitar . pasar a int, muchos nulos
arrival date of month, quita ., hace int
stays in weekend nights (num noches que el cliente se quedó durante el fin de semana) datos incongruentes?
STAYS_IN_WEEK_NIGHTS (lo mismo pero en la semana) también incongruente? valores flotantes, deberían ser int?
ADULTS cambiar .0 , dejar como int
children quitar puntos
babies quitar puntos
is repeated guest, cambiar o y 1 por true o false, y nan por undefined o similar
previous_cancellations quitar puntos . dejar como int
previous_bookings_not_canceled quita .0 dejar como int
booking_changes quitar .0 
agent quitar .0
company quitar .0
DAYS_IN_WAITING_LIST quitar .0
adr debería ser float , es int
REQUIRED_CAR_PARKING_SPACES quitar .0
total_of_special_requests quitar .0

In [44]:
#Lista de columnas float to integer
#float_to_int=['lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company', 'days_in_waiting_list', 'required_car_parking_spaces', 'total_of_special_requests']
#ok

#Lista de columnas Unknown
unknown=['hotel', 'is_canceled', 'country', 'market_segment', 'customer_type', 'reservation_status']
#Lista de columnas Undefined
undefined=['meal', 'distribution_channel']
#Lista de True/False object
true_false=['is_canceled', 'is_repeated_guest']
#True, False, 1, 0
#is_repeated_guest es float --> pasar primero a int y luego a T/F object??? 
#Lista fecha
objetc_to_date=['reservation_status_date']
#Lista eliminar
columns_to_drop=['0']
#Otros
#arrival_date_month --> hacer diccionario para renombrar los dígitos object en los nombres de los meses corespondientes
#reserved_room_type --> chequear si se puede cubrir con assigned_room_type
#assigned_room_type --> chequear si se puede curbir con reserved_room_type
#Sin modificaciones
#adr (tarifa diaria, already float)

# Paola


Lista de columnas Unknown:
* unknown=['hotel', 'is_canceled', 'country', 'market_segment', 'customer_type', 'reservation_status'] ✅

Lista de columnas Undefined
* undefined=['meal', 'distribution_channel']  ✅

Lista de True/False object
true_false=['is_canceled', 'is_repeated_guest']  ✅

True, False, 1, 0 ???

* is_repeated_guest es float --> pasar primero a int y luego a T/F object??? (revisar)... ✅❎

Lista fecha
* objetc_to_date=['reservation_status_date']  ✅
* he rrellenado los nulos de ['year'] usando el año de ['reservation_status_date'] (valorar)

#Lista eliminar
* columns_to_drop=['0','company'] ✅( company la agregue yo )

Otros
* arrival_date_month --> hacer diccionario para renombrar los dígitos object en los nombres de los meses corespondientes

* reserved_room_type --> chequear si se puede cubrir con assigned_room_type  ✅
* assigned_room_type --> chequear si se puede curbir con reserved_room_type  ✅

Sin modificaciones
* adr (tarifa diaria, already float)

----------------------------------------------------------------------------------------------------


|nombre columna |Nulos_Porcentaje| porque no??|
|---------------|----------------|------------|
|arrival_date_year|	4.97|  No se como llenarlo|
|arrival_date_week_number|	15.40| el numero de semana es variable y no se me ocurre como llenarlo|
|arrival_date_day_of_month |	0.10| creo que se podria dejar así, cambiandolo por algo que no nos sume|
|children|	41.98| numeros muy raros podriamos poner etiqueta de true y false, o no|
|previous_cancellations	|36.18| etiqueta???? 
|agent |	13.69 | no creo que se pueda rellenar con ninguna exepto si lo ha hecho por internet darle un valor|
|reservation_status_date	|10.82| no se como manejar los nulos de las fechas|

In [45]:
df_pao= df1.copy()

In [46]:
def llenar_nulos_con_unknown(df, lista):
    """
    Rellena los valores NaN en las columnas especificadas con 'unknown'.
    
    Args:
        df (pd.DataFrame): El DataFrame.
        lista (list): Lista de nombres de columnas a rellenar.
    
    Returns:
        pd.DataFrame: El DataFrame actualizado.
    """
    for col in lista:
        df[col].fillna('Unknown', inplace=True)
    return df

In [47]:
llenar_nulos_con_unknown(df_pao,['hotel', 'is_canceled', 'country', 'market_segment', 'customer_type', 'reservation_status'])

,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,Resort Hotel,False,342,2015,July,27,1,0,0,2,<NA>,0,BB,PRT,Unknown,Direct,False,<NA>,0,C,C,3,<NA>,<NA>,0,Transient,0.00,0,0,Check-Out,2015-07-01 00:00:00
1,1,Resort Hotel,False,737,<NA>,July,27,1,0,0,2,<NA>,0,BB,Unknown,Unknown,Direct,False,0,0,NaN,C,4,<NA>,<NA>,0,Transient,0.00,0,0,Check-Out,2015-07-01 00:00:00
2,2,Resort Hotel,False,7,2015,July,27,1,0,1,1,0,0,BB,GBR,Unknown,Direct,False,0,0,A,C,0,<NA>,<NA>,0,Transient,75.00,0,0,Check-Out,2015-07-02 00:00:00
3,3,Resort Hotel,False,13,<NA>,July,27,1,0,1,1,<NA>,0,BB,GBR,Corporate,Corporate,False,0,0,A,A,0,304,<NA>,0,Transient,75.00,0,0,Check-Out,2015-07-02 00:00:00
4,4,Resort Hotel,False,14,<NA>,July,<NA>,1,0,2,2,<NA>,0,BB,Unknown,Online TA,TA/TO,False,0,0,A,A,0,240,<NA>,0,Transient,98.00,0,1,Check-Out,2015-07-03 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,119385,City Hotel,False,23,2017,August,35,30,2,5,2,0,0,BB,Unknown,Offline TA/TO,TA/TO,False,0,0,A,A,0,394,<NA>,0,Unknown,96.14,0,0,Check-Out,2017-06-09 00:00:00
119386,119386,City Hotel,False,102,<NA>,August,35,31,2,5,3,<NA>,0,BB,FRA,Unknown,TA/TO,False,<NA>,0,E,E,0,9,<NA>,0,Transient,225.43,0,2,Check-Out,2017-07-09 00:00:00
119387,119387,City Hotel,False,34,2017,August,35,31,2,5,2,<NA>,0,BB,Unknown,Online TA,TA/TO,False,0,0,NaN,D,0,9,<NA>,0,Transient,157.71,0,4,Check-Out,2017-07-09 00:00:00
119388,119388,City Hotel,False,109,<NA>,August,35,31,2,5,55,0,0,BB,Unknown,Online TA,TA/TO,False,0,0,A,A,0,89,<NA>,0,Transient,104.40,0,0,Check-Out,2017-07-09 00:00:00


In [48]:
def llenar_nulos_con_undefined(df, lista):
    """
    Rellena los valores NaN en las columnas especificadas con 'unknown'.
    
    Args:
        df (pd.DataFrame): El DataFrame.
        lista (list): Lista de nombres de columnas a rellenar.
    
    Returns:
        pd.DataFrame: El DataFrame actualizado.
    """
    for col in lista:
        df[col].fillna('Undefined', inplace=True)
    return df

In [49]:
llenar_nulos_con_undefined(df_pao, ['meal', 'distribution_channel'])

,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,Resort Hotel,False,342,2015,July,27,1,0,0,2,<NA>,0,BB,PRT,Unknown,Direct,False,<NA>,0,C,C,3,<NA>,<NA>,0,Transient,0.00,0,0,Check-Out,2015-07-01 00:00:00
1,1,Resort Hotel,False,737,<NA>,July,27,1,0,0,2,<NA>,0,BB,Unknown,Unknown,Direct,False,0,0,NaN,C,4,<NA>,<NA>,0,Transient,0.00,0,0,Check-Out,2015-07-01 00:00:00
2,2,Resort Hotel,False,7,2015,July,27,1,0,1,1,0,0,BB,GBR,Unknown,Direct,False,0,0,A,C,0,<NA>,<NA>,0,Transient,75.00,0,0,Check-Out,2015-07-02 00:00:00
3,3,Resort Hotel,False,13,<NA>,July,27,1,0,1,1,<NA>,0,BB,GBR,Corporate,Corporate,False,0,0,A,A,0,304,<NA>,0,Transient,75.00,0,0,Check-Out,2015-07-02 00:00:00
4,4,Resort Hotel,False,14,<NA>,July,<NA>,1,0,2,2,<NA>,0,BB,Unknown,Online TA,TA/TO,False,0,0,A,A,0,240,<NA>,0,Transient,98.00,0,1,Check-Out,2015-07-03 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,119385,City Hotel,False,23,2017,August,35,30,2,5,2,0,0,BB,Unknown,Offline TA/TO,TA/TO,False,0,0,A,A,0,394,<NA>,0,Unknown,96.14,0,0,Check-Out,2017-06-09 00:00:00
119386,119386,City Hotel,False,102,<NA>,August,35,31,2,5,3,<NA>,0,BB,FRA,Unknown,TA/TO,False,<NA>,0,E,E,0,9,<NA>,0,Transient,225.43,0,2,Check-Out,2017-07-09 00:00:00
119387,119387,City Hotel,False,34,2017,August,35,31,2,5,2,<NA>,0,BB,Unknown,Online TA,TA/TO,False,0,0,NaN,D,0,9,<NA>,0,Transient,157.71,0,4,Check-Out,2017-07-09 00:00:00
119388,119388,City Hotel,False,109,<NA>,August,35,31,2,5,55,0,0,BB,Unknown,Online TA,TA/TO,False,0,0,A,A,0,89,<NA>,0,Transient,104.40,0,0,Check-Out,2017-07-09 00:00:00


In [50]:
df_pao['is_repeated_guest'].value_counts()# los valores de esta se pueden imputar con la moda, no encuentro una columna que me pueda ayudar a imputarlos de oytra manera????

is_repeated_guest
False    110730
<NA>       5014
True       3646
Name: count, dtype: int64


_¿Cuál es la tasa de cancelación de este hotel?

¿Ha cambiado la tasa de cancelación con el tiempo?

¿Se cancelan más las reservas de verano que de invierno?. ¿Se cancelan más las reservas de entre semana que las de los fines de semana?

Los clientes que cancelan, ¿presentan algún tipo de característica demográfica?

Las reservas que se hacen con mayor anticipación, ¿tienen mucho riesgo de cancelarse?

Las reservas que incluyen hijos, ¿tienen menor riesgo de cancelación?

Los usuarios que realizaron algún cambio en su reserva, ¿tienen menor riesgo de cancelación?

Cuándo el usuario ha realizado una solicitud especial, ¿el riesgo de cancelación es menor?

Las reservas que tienen un “adr” bajo, ¿tienen un riesgo menor de cancelación?

Relación con otras variables: Analizar cómo otras variables, como el tipo de comida incluida en la reserva o el número de solicitudes especiales, están relacionadas con las cancelaciones._

In [54]:
df2["adr"].unique()

array([  0.  ,  75.  ,  98.  , ..., 209.25, 157.71,    nan])

In [53]:
# ADR= Tarifa promedio diaria pagada por la reserva

df2["adr"].describe().T

count    119390.000000
mean        101.831122
std          50.535790
min          -6.380000
25%          69.290000
50%          94.575000
75%         126.000000
max        5400.000000
Name: adr, dtype: float64

In [71]:
df2["adr"]= df2["adr"].abs()

In [72]:
df2["adr"].describe().T

count    119390.000000
mean        101.831228
std          50.535575
min           0.000000
25%          69.290000
50%          94.575000
75%         126.000000
max        5400.000000
Name: adr, dtype: float64

In [68]:
filtrado = df2[(df2['adr'] > 0) & (df2['adr'] <= 20)]
#filtrado = df[df['nombre_de_la_columna'].between(0, 20)]

In [69]:
filtrado

,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
388,388,Resort Hotel,False,1.0,NaN,July,29.0,14.0,0.0,1.0,2.0,NaN,0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,2.0,240.0,NaN,0.0,NaN,8.0,0.0,1.0,Check-Out,2015-07-15 00:00:00
1227,1227,Resort Hotel,False,98.0,NaN,August,34.0,19.0,2.0,7.0,2.0,0.0,0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,D,D,0.0,147.0,NaN,0.0,Transient,8.0,0.0,0.0,Check-Out,2015-08-28 00:00:00
1382,1382,Resort Hotel,True,106.0,NaN,August,35.0,27.0,2.0,5.0,2.0,0.0,0,BB,NaN,NaN,TA/TO,0.0,NaN,0.0,A,A,0.0,240.0,NaN,0.0,Transient,8.0,0.0,1.0,Canceled,2015-05-13 00:00:00
1383,1383,Resort Hotel,True,108.0,2015.0,August,35.0,27.0,2.0,5.0,2.0,0.0,0,BB,NaN,NaN,TA/TO,0.0,0.0,0.0,NaN,A,0.0,240.0,NaN,0.0,Transient,8.0,0.0,2.0,Canceled,2015-08-14 00:00:00
1808,1808,Resort Hotel,False,0.0,2015.0,September,38.0,16.0,0.0,1.0,2.0,NaN,0,BB,ESP,NaN,Direct,0.0,NaN,0.0,NaN,C,0.0,NaN,NaN,0.0,Transient,8.0,1.0,0.0,Check-Out,2015-09-17 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115758,115758,City Hotel,False,65.0,NaN,July,28.0,10.0,1.0,1.0,2.0,2.0,0,BB,NaN,NaN,Direct,0.0,0.0,0.0,E,E,1.0,14.0,NaN,0.0,NaN,18.0,0.0,0.0,Check-Out,2017-12-07 00:00:00
115890,115890,City Hotel,False,274.0,2017.0,July,28.0,12.0,0.0,1.0,2.0,NaN,0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,1.0,9.0,NaN,0.0,Transient,15.0,0.0,2.0,Check-Out,2017-07-13 00:00:00
118200,118200,City Hotel,False,10.0,NaN,August,NaN,12.0,2.0,2.0,0.0,NaN,0,BB,MAR,Direct,Direct,0.0,0.0,0.0,NaN,A,1.0,NaN,NaN,0.0,Transient-Party,6.0,0.0,1.0,Check-Out,2017-08-16 00:00:00
118554,118554,City Hotel,False,59.0,NaN,August,34.0,20.0,2.0,0.0,1.0,0.0,0,BB,NaN,NaN,TA/TO,0.0,0.0,0.0,A,A,1.0,98.0,NaN,0.0,Transient-Party,6.0,0.0,0.0,Check-Out,2017-08-22 00:00:00


In [52]:
# Llenar los valores nulos con False (la moda)
df_pao['is_repeated_guest']= df_pao['is_repeated_guest'].astype(str)

df_pao['is_repeated_guest']= df_pao['is_repeated_guest'].replace('<NA>', 0 ).replace('False',0).replace('True',1)

# Convertir la columna a tipo booleano

df_pao['is_repeated_guest'] = df_pao['is_repeated_guest'].astype(bool)

In [53]:
df_pao['is_repeated_guest'].value_counts()

False    115744
True       3646
Name: is_repeated_guest, dtype: int64

In [54]:
def convertir_columnas_a_entero(df, lista_columnas):
    """
    Convierte las columnas especificadas a tipo entero (int).

    Args:
        df (pd.DataFrame): El DataFrame.
        lista_columnas (list): Lista de nombres de columnas a convertir.

    Returns:
        pd.DataFrame: El DataFrame actualizado.
    """
    for col in lista_columnas:
        df[col] = df[col].astype(int)
    return df



In [55]:
# # Convertir la columna a tipo datetime
# df_pao['reservation_status_date'] = pd.to_datetime(df_pao['reservation_status_date'])

# # Extraer solo la fecha
# df_pao['reservation_status_date'] = df_pao['reservation_status_date'].dt.date

# error:day is out of range for month: 2030-02-30

In [56]:
# Convertir las fechas a tipo datetime con manejo de errores
df_pao['reservation_status_date'] = pd.to_datetime(df_pao['reservation_status_date'], errors='coerce') # errors='coerce' para convertir las fechas inválidas a valores nulos (NaN)

# Extraer solo la fecha válida
df_pao['reservation_status_date'] = df_pao['reservation_status_date'].dt.date

# Convertir en tipo date:
df_pao['reservation_status_date'] = pd.to_datetime(df_pao['reservation_status_date'])

df_pao['reservation_status_date']



0        2015-07-01
1        2015-07-01
2        2015-07-02
3        2015-07-02
4        2015-07-03
            ...    
119385   2017-06-09
119386   2017-07-09
119387   2017-07-09
119388   2017-07-09
119389   2017-07-09
Name: reservation_status_date, Length: 119390, dtype: datetime64[ns]

In [57]:
df_pao['reservation_status_date']

0        2015-07-01
1        2015-07-01
2        2015-07-02
3        2015-07-02
4        2015-07-03
            ...    
119385   2017-06-09
119386   2017-07-09
119387   2017-07-09
119388   2017-07-09
119389   2017-07-09
Name: reservation_status_date, Length: 119390, dtype: datetime64[ns]

In [58]:
#Extraer el año solo de las fechas no nulas

df_pao['arrival_date_year'].fillna(df_pao['reservation_status_date'].dt.year, inplace=True)

In [59]:
df_pao['arrival_date_year'].unique()

<IntegerArray>
[2015, <NA>, 2016, 2017, 2014]
Length: 5, dtype: Int64

In [60]:
df_pao.tail(20)

,Unnamed: 0,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
119370,119370,City Hotel,False,201,2017,August,35,30,1,4,1,0,0,BB,Unknown,Unknown,TA/TO,False,0,0,A,A,0,9,<NA>,0,Transient,108.00,0,2,Check-Out,2017-04-09
119371,119371,City Hotel,False,175,2017,August,35,31,1,3,2,<NA>,0,BB,NLD,Unknown,TA/TO,False,0,0,A,A,1,42,<NA>,0,Transient,86.85,0,2,Check-Out,2017-04-09
119372,119372,City Hotel,False,175,2017,August,35,31,1,3,51,0,0,BB,Unknown,Offline TA/TO,TA/TO,False,<NA>,0,A,A,0,42,<NA>,0,Transient,82.35,0,1,Check-Out,2017-04-09
119373,119373,City Hotel,False,175,2017,August,35,31,1,3,1,0,0,BB,Unknown,Offline TA/TO,TA/TO,False,<NA>,0,A,A,0,42,<NA>,0,Transient,82.35,0,1,Check-Out,2017-04-09
119374,119374,City Hotel,False,201,2017,August,35,29,1,5,2,<NA>,0,BB,DEU,Unknown,TA/TO,False,<NA>,0,A,A,0,9,<NA>,0,Transient,117.00,0,2,Check-Out,2017-04-09
119375,119375,City Hotel,False,200,2017,August,35,30,1,4,2,0,0,BB,AUT,Unknown,TA/TO,False,0,0,A,A,0,16,<NA>,0,Transient,85.50,0,1,Check-Out,2017-04-09
119376,119376,City Hotel,False,165,2017,August,<NA>,29,1,5,3,0,0,BB,AUT,Unknown,TA/TO,False,0,0,NaN,E,0,9,<NA>,0,Unknown,209.25,0,3,Check-Out,2017-04-09
119377,119377,City Hotel,False,185,2017,August,35,30,1,4,2,0,0,SC,CHE,Unknown,TA/TO,False,<NA>,0,A,A,0,9,<NA>,0,Transient,99.00,0,1,Check-Out,NaT
119378,119378,City Hotel,False,247,2017,August,<NA>,31,1,3,2,0,0,BB,GBR,Unknown,TA/TO,False,0,0,NaN,A,0,42,<NA>,0,Transient,86.85,0,0,Check-Out,2017-04-09
119379,119379,City Hotel,False,109,2017,August,35,31,1,3,50,<NA>,0,BB,GBR,Online TA,TA/TO,False,0,0,D,D,0,9,<NA>,0,Unknown,155.00,0,1,Check-Out,2017-04-09


In [61]:
df_pao['arrival_date_month'].unique()

array(['July', 'August', 'September', 'October', 'November', 'December',
       'January', 'February', 'March', 'April', 'May', 'June'],
      dtype=object)

In [62]:
df_pao['assigned_room_type'].unique()

array(['C', 'A', 'D', 'E', 'G', 'F', 'I', 'B', 'H', 'P', 'L', 'K'],
      dtype=object)

In [69]:
df_pao['assigned_room_type'].fillna(df_pao['reserved_room_type'], inplace=True)

In [63]:
df_pao['reserved_room_type'].fillna(df_pao['assigned_room_type'], inplace=True)

In [74]:
# porcentajes de nulos
porcentaje_null= (df_pao.isnull().sum()/df_pao.shape[0]) *100 # cálculo
nulos_pao= pd.DataFrame(porcentaje_null.round(2), columns = ["Nulos_Porcentaje"]) #convierto a dataframe y redondeo las cifras
display(nulos_pao[nulos_pao["Nulos_Porcentaje"]>0]) #filtro para que solo me muestre las columnas con más de cero nulos.

,Nulos_Porcentaje
arrival_date_year,4.97
arrival_date_week_number,15.40
arrival_date_day_of_month,0.10
children,41.98
previous_cancellations,36.18
agent,13.69
reservation_status_date,10.82


In [71]:
df_pao['children'].value_counts().sum() # etiquetas true and false / numeros de mas de 5

69270

In [ ]:
# company muchos nulos 
df_pao = df_pao.drop( 'company', axis=1)

In [73]:
# df_pao.to_csv('df_limpieza_Ra_+_pao.csv')